In [1]:
using Random
using Distributions
using LinearAlgebra
using ForwardDiff
include("../Inversion/Plot.jl")
include("../Inversion/GMGD.jl")

visualization_2d (generic function with 1 method)

## Gaussian mixture

This is not an inverse problem!

$$
e^{-\Phi_r(\theta)} \propto \sum_i w_i \mathcal{N}(\theta; m_i, C_i) \\
\Phi_r(\theta) = - \log\Bigl( \sum_i w_i \mathcal{N}(\theta; m_i, C_i) \Bigr)
$$



In [30]:
function Gaussian_mixture_component(x, args)
    _, x_mean, inv_sqrt_xx_cov = args
    # C = L L.T
    # C^-1 = L^-TL^-1
    N_modes, N_x = size(x_mean)
    ρ = zeros(N_modes)
    for im = 1:N_modes
        ρ[im] = exp(-0.5*(x-x_mean[im,:])'*(inv_sqrt_xx_cov[im]'*inv_sqrt_xx_cov[im]*(x-x_mean[im,:])))/abs(det(inv_sqrt_xx_cov[im]))/(2*π)^(N_x/2)
    end
    return ρ
end


function Gaussian_mixture_logrho(x, args)
    x_w, x_mean, inv_sqrt_xx_cov = args
    # C = L L.T
    # C^-1 = L^-TL^-1
    N_x = size(x_mean, 2)
    ρ = 0
    for im = 1:length(x_w)
        ρ += x_w[im]*exp(-0.5*(x-x_mean[im,:])'*(inv_sqrt_xx_cov[im]'*inv_sqrt_xx_cov[im]*(x-x_mean[im,:])))/abs(det(inv_sqrt_xx_cov[im]))
    end
    return log( ρ ) - N_x/2*log(2*π)
end



function Gaussian_mixture_dlogrho(x, args)
    return Gaussian_mixture_logrho(x, args), 
           ForwardDiff.gradient(x -> Gaussian_mixture_logrho(x, args), x), 
           ForwardDiff.hessian(x -> Gaussian_mixture_logrho(x, args), x)
end


function Gaussian_mixture_integral(x_lims, Ns, args)
    x_w, x_mean, inv_sqrt_xx_cov = args
    N_x = size(x_mean, 2)
    ρlogρ   = zeros(N_modes, Ns...)
    ρ∇logρ  = zeros(N_modes, N_x, Ns...)
    ρ∇²logρ = zeros(N_modes, N_x, N_x, Ns...)
    
    
    
    dim = length(Ns)
    xx = range(x_lims[1][1], stop=x_lims[1][2], length=Ns[1]); dx = xx[2] - xx[1]
        
    if dim == 1
        
        for i = 1:Ns[1]
            ρ = Gaussian_mixture_component(xx[i], args)
            logρ, ∇logρ, ∇²logρ = Gaussian_mixture_dlogrho(xx[i], args)
            for im = 1:N_modes
                    ρlogρ[im,i], ρ∇logρ[im,:,i], ρ∇²logρ[im,:,:,i] = ρ[im]*logρ, ρ[im]*∇logρ, ρ[im]*∇²logρ
            end
        end
        ρlogρ_mean, ρ∇logρ_mean, ρ∇²logρ_mean = sum(ρlogρ, dims=2)*dx, sum(ρ∇logρ, dims=3)*dx, sum(ρ∇²logρ, dims=4)*dx
    
    else
        
        yy = range(x_lims[2][1], stop=x_lims[2][2], length=Ns[2]); dy = yy[2] - yy[1]
        
        for i = 1:Ns[1]
            for j = 1:Ns[2]
                ρ = Gaussian_mixture_component([xx[i];yy[j]], args)
                logρ, ∇logρ, ∇²logρ = Gaussian_mixture_dlogrho([xx[i];yy[j]], args)
                for im = 1:N_modes
                    ρlogρ[im,i,j], ρ∇logρ[im,:,i,j], ρ∇²logρ[im,:,:,i,j] = ρ[im]*logρ, ρ[im]*∇logρ, ρ[im]*∇²logρ
                end
            end
        end
        ρlogρ_mean, ρ∇logρ_mean, ρ∇²logρ_mean = sum(ρlogρ, dims=[2,3])*dx*dy, sum(ρ∇logρ, dims=[3,4])*dx*dy, sum(ρ∇²logρ, dims=[4,5])*dx*dy
    
    end
    
    return ρlogρ_mean, ρ∇logρ_mean, ρ∇²logρ_mean
    
end


Gaussian_mixture_integral (generic function with 1 method)

In [36]:
# Gaussian mixture
N_modes = 2
x_w, x_mean = [0.2;0.8], [1.0 2.0; 3.0 1.0] 

N_modes, N_x = size(x_mean)

xx_cov = zeros(N_modes, 2, 2)
for im = 1:N_modes
    xx_cov[im,:,:] += I
end

x_lims, Ns = [[minimum(x_mean[1,:])-5; maximum(x_mean[1,:])+5], [minimum(x_mean[2,:])-5, maximum(x_mean[2,:])+5]], [500,500]

for compute_sqrt_matrix_type in ["Cholesky", "SVD"]
    sqrt_xx_cov, inv_sqrt_xx_cov = [], []
    for im = 1:N_modes
        sqrt_cov, inv_sqrt_cov = compute_sqrt_matrix(xx_cov[im,:,:]; type=compute_sqrt_matrix_type) 
        push!(sqrt_xx_cov, sqrt_cov)
        push!(inv_sqrt_xx_cov, inv_sqrt_cov) 
    end
    
    # compute reference
    args = (x_w, x_mean, inv_sqrt_xx_cov)
    ρlogρ_mean_ref, ρ∇logρ_mean_ref, ρ∇²logρ_mean_ref = Gaussian_mixture_integral(x_lims, Ns, args)

    c_weight = 0.1 

    for quadrature_type in ["cubature_transform_o3", "cubature_transform_o5", "unscented_transform", "mean_point"]
        _, c_weights_GM, mean_weights_GM = generate_quadrature_rule(N_x, quadrature_type; c_weight=c_weight)
        logρ_mean, ∇logρ_mean, ∇²logρ_mean = compute_logρ_gm_expectation(x_w, x_mean, sqrt_xx_cov, inv_sqrt_xx_cov, c_weights_GM, mean_weights_GM)
        
        @info "qaudrature type = ", quadrature_type, " ##############################"
        @info "ρlogρ error = ", norm(logρ_mean - ρlogρ_mean_ref), " / ", norm(ρlogρ_mean_ref)
        @info "ρ∇logρ error = ", norm(∇logρ_mean - ρ∇logρ_mean_ref), " / ", norm(ρ∇logρ_mean_ref)
        @info "ρ∇²logρ error = ", norm(∇²logρ_mean - ρ∇²logρ_mean_ref), " / ", norm(ρ∇²logρ_mean_ref)
    end
end


[ Info: ("qaudrature type = ", "cubature_transform_o3", " ##############################")
[ Info: ("ρlogρ error = ", 0.04446712923564575, " / ", 4.786664196658497)
[ Info: ("ρ∇logρ error = ", 0.07147648098845928, " / ", 0.7993108498824462)
[ Info: ("ρ∇²logρ error = ", 0.10044221881246984, " / ", 1.6288719243465815)
[ Info: ("qaudrature type = ", "cubature_transform_o5", " ##############################")
[ Info: ("ρlogρ error = ", 0.010203319923316344, " / ", 4.786664196658497)
[ Info: ("ρ∇logρ error = ", 0.03892244581725066, " / ", 0.7993108498824462)
[ Info: ("ρ∇²logρ error = ", 0.021176836614785338, " / ", 1.6288719243465815)
[ Info: ("qaudrature type = ", "unscented_transform", " ##############################")
[ Info: ("ρlogρ error = ", 0.09356068773730782, " / ", 4.786664196658497)
[ Info: ("ρ∇logρ error = ", 0.3044183455122153, " / ", 0.7993108498824462)
[ Info: ("ρ∇²logρ error = ", 0.045699741538079736, " / ", 1.6288719243465815)
[ Info: ("qaudrature type = ", "mean_point", "